In [4]:
import os

In [1]:
%pwd

'e:\\MyLearning\\Brain_Tumor\\Brain_Tumor\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'e:\\MyLearning\\Brain_Tumor\\Brain_Tumor'

In [7]:
#initializing the entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
#creating configuration manager
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
                 self.config = read_yaml(str(config_filepath))
                 self.params = read_yaml(str(params_filepath))
                 create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self)->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config

In [14]:
#creating components

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [15]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"run_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    def _create_model_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = str(self.config.checkpoint_model_filepath),
            save_best_only = True
        )
        
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_model_ckpt_callbacks
        ]

In [16]:
#pipeline

try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callbacks_config()
    prepare_callback = PrepareCallback(prepare_callback_config)
    callback_list = prepare_callback.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2025-01-29 15:16:29,612: INFO: common: Reading the yaml file: config\config.yaml]
[2025-01-29 15:16:29,617: INFO: common: Reading the yaml file: params.yaml]
[2025-01-29 15:16:29,620: INFO: common: Directory already exists at: artifacts]
[2025-01-29 15:16:29,624: INFO: common: Directory created at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-01-29 15:16:29,627: INFO: common: Directory created at: artifacts\prepare_callbacks\tensorboard_log_dir]
